# Session 9 homework

In this homework, we'll deploy the dogs vs cats model we trained in the previous homework.

Download the model from here:

In [ ]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats-model/dogs_cats_10_0.687.h5

# Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
model = keras.models.load_model('dogs_cats_10_0.687.h5')

2021-11-26 00:11:24.900138: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

model_lite = converter.convert()

2021-11-26 00:11:26.735476: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/z6/wchwkzy93yqb85l4qc39505r0000gn/T/tmpl_cmpudl/assets


2021-11-26 00:11:27.463179: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-11-26 00:11:27.463257: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-11-26 00:11:27.464201: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.

2021-11-26 00:11:28.210069: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-11-26 00:11:28.210085: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-11-26 00:11:28.306809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:210] disabling MLIR crash reproducer, set env var `MLIR

In [4]:
with open('model.tflite', 'wb') as f_out:
    f_out.write(model_lite)

In [5]:
import os
filesize = os.path.getsize('model.tflite')
print(f'{ filesize / (1024**2)} MB')

42.787506103515625 MB


## Question 1 answer

Around `43` MB in size.

# Question 2

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

In [7]:
import tensorflow.lite as tflite

In [8]:
interpreter = tflite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

INFO: Initialized TensorFlow Lite runtime.


In [9]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print(output_index)

13


## Question 2 Answer

The output index for the lite model is `13`.

# Preparing the image

You'll need some code for downloading and resizing images. You'll need pillow:

In [10]:
'''
!pip install pillow
'''

'\n!pip install pillow\n'

Use the following code:

In [11]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

Let's download and resize this image:

In [ ]:
'''
OH GOD WHAT AM I DOING
DON'T USE THIS BLOCK YOU IDIOT
!wget https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg
'''

In [12]:
'''
USE THIS BLOCK INSTEAD
'''
img = prepare_image(
    download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'),
    (150, 150)
)

Based on [the solution of the previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/08-deep-learning/CNN_solution.ipynb), what should be the target size for the image?

* `150x150`

# Question 3

Now we need to turn the image into an numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [ ]:
'''
Since we do not have to train the network, we do not need data augmentation.

The main pre-processing is image resizing and rescaling, then.

target_size=(150,150)
rescale=1./255
dtype=float32
'''

'''
DON'T USE THIS BLOCK

from tensorflow.keras.preprocessing.image import load_img

img = load_img('Pug_600.jpg', target_size=(150,150))
x = np.array(img, dtype='float32')
x = x/ 255

x[0,0,0]
'''

In [ ]:
'''
ALTERNATIVE VERSION
No TF dependency

DON'T USE THIS BLOCK EITHER

from PIL import Image

with Image.open('Pug_600.jpg') as img:
    img = img.resize((150, 150), Image.NEAREST)
x = np.array(img, dtype='float32')
x = x/ 255

x[0,0,0]
'''

In [13]:
'''
USE ME INSTEAD
'''

x = np.array(img, dtype='float32')
x = x/ 255

x[0,0,0]

0.7058824

## Question 3 answer

The value of the first pixel is `0.7058824`

# Question 4

Now let's apply this model to this image. What's the output of the model?

In [14]:
# batchify
X = np.array([x])

In [15]:
interpreter.set_tensor(input_index, X)

In [16]:
interpreter.invoke()

In [17]:
preds = interpreter.get_tensor(output_index)
preds

array([[0.7704913]], dtype=float32)

## Question 4 answer

The output of the model is `0.7704913`.

# Prepare the Lambda code

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook by importing the file and invoking the function from this file.

(Continued in `homework2.ipynb`)